In [ ]:
# !pip install google-cloud-storage
# !pip install firebase_admin
# !pip install SpeechRecognition
# !pip install pyaudio

In [1]:
def getDict(ans,samples):
  prescription = {}
  name = ""
  age = None
  gender = None
  disease = ""
  symptom_duration = None
  symptoms = {}
  advise = {}
  num_medic = 0
  num_du = 0
  num_dof = 0
  num_do = 0
  symp_start = ""
  adv_start = ""
  medi = []
  du = []
  do = []
  dof = []  
  medi_start = ""
  du_start = ""
  dof_start = ""
  do_start = ""

  for i in range(len(ans)):
    if ans[i] == "B-NA":
      name = samples.split()[i]
    elif ans[i] == "I-NA":
      name = name + " " + samples.split()[i]

    if ans[i] == "B-DI":
      disease = samples.split()[i]
    elif ans[i] == "I-DI":
      disease += " " + samples.split()[i]


    if ans[i] == "B-SYDU":
      symptom_duration = samples.split()[i]
    elif ans[i] == "I-SYDU":
      symptom_duration += " " + samples.split()[i]


    if ans[i] == "B-AG":
      age = samples.split()[i]
    if ans[i] == "B-GE":
      gender = samples.split()[i]

    if ans[i] == "B-ME":
      num_medic += 1
    if ans[i] == "B-DO":
      num_do += 1
    if ans[i] == "B-DU":
      num_du += 1
    if ans[i] == "B-DOF":
      num_dof += 1

    if ans[i] == "B-SY" and ans[i+1] == "I-SY":
      symp_start += samples.split()[i]
    elif ans[i] == "B-SY" and ans[i+1] != "I-SY":
      symptoms[samples.split()[i]] = True
    elif ans[i] == "I-SY" and ans[i+1] == "I-SY":
      symp_start += " " + samples.split()[i]
    elif ans[i] == "I-SY" and ans[i+1] != "I-SY":
      symp_start += " " + samples.split()[i]
      symptoms[symp_start] = True
      symp_start = ""

    if ans[i] == "B-AD" and ans[i+1] == "I-AD":
      adv_start += samples.split()[i]
    elif ans[i] == "B-AD" and ans[i+1] != "I-AD":
      advise[samples.split()[i]] = True
    elif ans[i] == "I-AD" and ans[i+1] == "I-AD":
      adv_start += " " + samples.split()[i]
    elif ans[i] == "I-AD" and ans[i+1] != "I-AD":
      adv_start += " " + samples.split()[i]
      advise[adv_start] = True
      adv_start = ""

    if ans[i] == "B-ME" and ans[i+1] == "I-ME":
      medi_start += samples.split()[i]
    elif ans[i] == "B-ME" and ans[i+1] != "I-ME":
      medi.append(samples.split()[i])
    elif ans[i] == "I-ME" and ans[i+1] == "I-ME":
      medi_start += " " + samples.split()[i]
    elif ans[i] == "I-ME" and ans[i+1] != "I-ME":
      medi_start += " " + samples.split()[i]
      medi.append(medi_start)
      medi_start = ""

    
    if ans[i] == "B-DU" and ans[i+1] == "I-DU":
      du_start += samples.split()[i]
    elif ans[i] == "B-DU" and ans[i+1] != "I-DU":
      du.append(samples.split()[i])
    elif ans[i] == "I-DU" and ans[i+1] == "I-DU":
      du_start += " " + samples.split()[i]
    elif ans[i] == "I-DU" and ans[i+1] != "I-DU":
      du_start += " " + samples.split()[i]
      du.append(du_start)
      du_start = ""

    if ans[i] == "B-DOF" and ans[i+1] == "I-DOF":
      dof_start += samples.split()[i]
    elif ans[i] == "B-DOF" and ans[i+1] != "I-DOF":
      dof.append(samples.split()[i])
    elif ans[i] == "I-DOF" and ans[i+1] == "I-DOF":
      dof_start += " " + samples.split()[i]
    elif ans[i] == "I-DOF" and ans[i+1] != "I-DOF":
      dof_start += " " + samples.split()[i]
      dof.append(dof_start)
      dof_start = ""

    if ans[i] == "B-DO" and ans[i+1] == "I-DO":
      do_start += samples.split()[i]
    elif ans[i] == "B-DO" and ans[i+1] != "I-DO":
      do.append(samples.split()[i])
    elif ans[i] == "I-DO" and ans[i+1] == "I-DO":
      do_start += " " + samples.split()[i]
    elif ans[i] == "I-DO" and ans[i+1] != "I-DO":
      do_start += " " + samples.split()[i]
      do.append(do_start)
      do_start = ""


  if gender == None:
    if "He" in samples.split() or "he" in samples.split():
      gender = "male"
    elif "She" in samples.split() or "she" in samples.split():
      gender = "female"
 

  prescription["name"] = name
  prescription["age"] = age
  prescription["gender"] = gender
  prescription["symptoms"] = symptoms
  prescription["disease"] = disease
  prescription["advise"] = advise
  prescription["symptom_duration"] = symptom_duration

  prescription["medication"] = {}
  for m in medi:
    prescription["medication"][m] = {}

  if num_medic == num_dof:
    for i in range(num_medic):
      prescription["medication"][medi[i]]["dosage_frequency"] = dof[i]
  elif num_medic != num_dof and num_dof!=0:
    for i in range(num_medic):
      prescription["medication"][medi[i]]["dosage_frequency"] = dof[0]

  if num_medic == num_du:
    for i in range(num_medic):
      prescription["medication"][medi[i]]["duration"] = du[i]
  elif num_medic != num_du and len(du)!=0:
    for i in range(num_medic):
      prescription["medication"][medi[i]]["duration"] = du[0]

  helper = {}
  if num_medic == num_do:
    for i in range(num_medic):
      prescription["medication"][medi[i]]["dosage"] = do[i]
  else:
    for i in range(len(ans)):
      if ans[i]=='B-DO':
        for j in range(i-3,i+3):
          if ans[j]=='B-ME':
            helper[samples.split()[i]] = j

    for i in range(len(do)):
      for j in range(len(medi)):
        if helper.get(do[i].split()[0]) and samples.split()[helper[do[i].split()[0]]] in medi[j]:
          prescription["medication"][medi[j]]["dosage"] = do[i]
  return prescription
  

In [2]:
def convertDict(nd):
    i = 1
    symp = {}
    adv = {}
    med = {}
    for sy in nd.get('symptoms').keys():
        symp['Symptom ' + str(i)] = sy
        i += 1
    i = 1
    for ad in nd.get('advise').keys():
        adv['Advice ' + str(i)] = ad
        i += 1
    i = 1
    for md in nd.get('medication').keys():
        med['Medication ' + str(i)] = {
            md: nd.get('medication').get(md)
        }
        i += 1
        
    return {
        'name': {'name': nd.get('name')},
        'age': {'age': nd.get('age')},
        'gender': {'gender': nd.get('gender')},
        'symptoms': symp,
        'disease': {'disease': nd.get('disease')},
        'advice': adv,
        'medication': med
    }
    

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("prescriptions.csv", encoding="latin1")
data = data.fillna(method="ffill")

words = list(set(data["Word"].values))
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words)

tags = list(set(data["Tag"].values))

n_tags = len(tags)

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


getter = SentenceGetter(data)
sent = getter.get_next()
sentences = getter.sentences

max_len = 90

i = 0
word2idx = {}
for w in words:
  if word2idx.get(w.lower()) or word2idx.get(w.capitalize()):
    continue
  else:
    if w == "ENDPAD":
      continue
    word2idx[w.lower()]=i
    word2idx[w.capitalize()]=i
    i+=1

tag2idx = {t: i for i, t in enumerate(tags)}
word2idx["ENDPAD"]=n_words-1

inverted_dict = {value: key for key, value in word2idx.items()}


In [4]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=40, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=15, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model) 

model = Model(input, out)
model.summary()
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X, np.array(y),batch_size=4, epochs=100, validation_split=0.1, verbose=1)

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 90)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 90, 40)            23000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 90, 40)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 90, 30)            6720      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 90, 22)            682       
Total params: 30,402
Trainable params: 30,402
Non-trainable params: 0
_________________________________________________________________


C:\Users\RAM\Anaconda3\IOSDPython\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 82 samples, validate on 10 samples
Epoch 1/100
82/82 [==============================] - 5s 64ms/step - loss: 2.5449 - accuracy: 0.6298 - val_loss: 1.8141 - val_accuracy: 0.6756
Epoch 2/100
82/82 [==============================] - 3s 39ms/step - loss: 1.5224 - accuracy: 0.7137 - val_loss: 1.5334 - val_accuracy: 0.6756
Epoch 3/100
82/82 [==============================] - 4s 51ms/step - loss: 1.3282 - accuracy: 0.7137 - val_loss: 1.3408 - val_accuracy: 0.6756
Epoch 4/100
82/82 [==============================] - 4s 53ms/step - loss: 1.2004 - accuracy: 0.7137 - val_loss: 1.2378 - val_accuracy: 0.6756
Epoch 5/100
82/82 [==============================] - 4s 50ms/step - loss: 1.1169 - accuracy: 0.7137 - val_loss: 1.1683 - val_accuracy: 0.6756
Epoch 6/100
82/82 [==============================] - 4s 53ms/step - loss: 1.0450 - accuracy: 0.7138 - val_loss: 1.1087 - val_accuracy: 0.6756
Epoch 7/100
82/82 [==============================] - 4s 52ms/step - loss: 0.9755 - accuracy: 0.7142 - va

82/82 [==============================] - 4s 53ms/step - loss: 0.0381 - accuracy: 0.9930 - val_loss: 0.1748 - val_accuracy: 0.9511
Epoch 59/100
82/82 [==============================] - 4s 50ms/step - loss: 0.0340 - accuracy: 0.9938 - val_loss: 0.1697 - val_accuracy: 0.9522
Epoch 60/100
82/82 [==============================] - 4s 49ms/step - loss: 0.0325 - accuracy: 0.9939 - val_loss: 0.1684 - val_accuracy: 0.9544
Epoch 61/100
82/82 [==============================] - 4s 53ms/step - loss: 0.0299 - accuracy: 0.9947 - val_loss: 0.1668 - val_accuracy: 0.9533
Epoch 62/100
82/82 [==============================] - 5s 59ms/step - loss: 0.0292 - accuracy: 0.9946 - val_loss: 0.1660 - val_accuracy: 0.9556
Epoch 63/100
82/82 [==============================] - 4s 51ms/step - loss: 0.0289 - accuracy: 0.9946 - val_loss: 0.1660 - val_accuracy: 0.9578
Epoch 64/100
82/82 [==============================] - 4s 54ms/step - loss: 0.0278 - accuracy: 0.9953 - val_loss: 0.1620 - val_accuracy: 0.9578
Epoch 65/100

In [5]:
# !pip install google-cloud-storage

In [6]:
# !pip install firebase_admin

In [5]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Fetch the service account key JSON file contents
# cred = credentials.Certificate('fir-189ce-firebase-adminsdk-e368z-ddbed10d26.json')
cred = credentials.Certificate('imagetotextf-firebase-adminsdk-tii7k-544931e7be.json')
# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://imagetotextf.firebaseio.com/'
})




In [6]:
import urllib
import json
def download_file(my_url):
    
    # my_url = "https://firebasestorage.googleapis.com/v0/b/fir-189ce.appspot.com/o/media.io_WhatsApp%20Audio%202020-01-03%20at%203.53.36%20PM.wav?alt=media&token=ee94b151-0014-4c30-9601-8b3208cf4b87"

    try:
        loader = urllib.request.urlretrieve(my_url, "prescription.wav")
        return loader
    except urllib.error.URLError as e:
        message = json.loads(e.read())
        print(message["error"]["message"])
    else:
        print(loader)

In [ ]:
refFile = db.reference('FilePath')
refpython = db.reference('python')

In [10]:
# !pip install SpeechRecognition

In [11]:
# !pip install pyaudio

In [ ]:
import speech_recognition as sr
import pyaudio
r = sr.Recognizer()

In [ ]:
nd = {}

In [ ]:
while True:
    
    if refFile.get()['URL']:
        d = refFile.get()
        urlLink = d['URL']
        download_file(urlLink)
        with sr.AudioFile('prescription.wav') as source:
          audio = r.listen(source)
        text = r.recognize_google(audio)
        print(text)

        s = ""
        spl = text.split()
        c = len(spl)
        spl2 = []
        for i in range(c):
#             print(len(spl2),len(spl),sep = " ")
            if spl[i] == "mg" or spl[i] == "ml":
                spl2[len(spl2)-1] += spl[i]
            elif spl[i] == "MG" or spl[i] == "ML":
                spl2[len(spl2)-1] += spl[i].lower()
            else:
                spl2.append(spl[i])
        text = spl2
        for a in text:
          if word2idx.get(a):
            if s == "":
              s += a 
            else:
              s += " " + a 
        print(s)
        lis = [x for x in s.split()]
        for i in range(len(s.split()),90):
          lis.append('ENDPAD')
#         print(lis)
        test = [word2idx[w] for w in lis]
        p = model.predict(np.array([test]))
        p = np.argmax(p, axis=-1)
        ans = [tags[x] for x in p[0]]
        for x,y in zip(lis,ans):
            print(x + " - " + y,end="\n")
        print(ans)
        d['URL'] = False
        refFile.set(d)
        
        dp = getDict(ans,s)
        nd = dp
        refpython.set({'isAvailable':convertDict(dp)})

In [20]:
model.save('savingmodel.h5')
model.save_weights('savingweights.h5')